# 📅 LoadStaticDimensions - Dates and Times

This notebook loads and transforms the **static dimensions** (`dates` and `times`) through the Medallion architecture:

- **Raw → Bronze**: CSV files are read from the Raw layer and stored as Delta tables in Bronze.  
- **Bronze → Silver**: typing transformations (`cast`) are applied using dedicated functions (`silver_transform_dates`, `silver_transform_times`).  
- **Silver → Gold**: the final dimensional tables (`DimDates`, `DimTimes`) are created and made ready for the analytical model.  
 


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import DateType, IntegerType, StringType, TimestampType

- `src_values`: list of static dimensions to process (`dates`, `times`).  
- `raw_base_path`, `bronze_base_path`, `silver_base_path`, `gold_base_path`: base paths for each layer in the Medallion architecture.  
- `pk_map`: maps each dimension to its primary key (`date_sk`, `time_sk`).  
- `bronze_schema`, `silver_schema`, `gold_schema`: schemas where the tables will be stored in each layer.  

In [0]:
src_values = ["dates", "times"]  
raw_base_path = "/Volumes/workspace/raw/rawvolume/rawdata"
bronze_base_path = "/Volumes/workspace/bronze/bronzevolume"
silver_base_path = "/Volumes/workspace/silver"
gold_base_path = "/Volumes/workspace/gold"
pk_map = {
    "dates": "date_sk",
    "times": "time_sk"
}
bronze_schema = "workspace.bronze"
silver_schema = "workspace.silver"
gold_schema   = "workspace.gold"

- **`silver_transform_dates(df)`**: Converts dates and related columns to proper types (date and int).

- **`silver_transform_times(df)`**: Converts time keys to int and keeps time attributes as strings. 

In [0]:
def silver_transform_dates(df):
    return (df.withColumn("full_date", F.col("full_date").cast(DateType()))
              .withColumn("date_sk", F.col("date_sk").cast(IntegerType()))
              .withColumn("year", F.col("year").cast(IntegerType()))
              .withColumn("month", F.col("month").cast(IntegerType()))
              .withColumn("day", F.col("day").cast(IntegerType()))
              .withColumn("weekday", F.col("weekday").cast(IntegerType()))
              .withColumn("quarter", F.col("quarter").cast(IntegerType())))

def silver_transform_times(df):
    return (df.withColumn("time_sk", F.col("time_sk").cast(IntegerType()))
              .withColumn("full_time", F.col("full_time").cast(StringType()))
              .withColumn("hour", F.col("hour").cast(IntegerType()))
              .withColumn("minute", F.col("minute").cast(IntegerType()))
              .withColumn("day_part", F.col("day_part").cast(StringType()))
              .withColumn("full_time_ampm", F.col("full_time_ampm").cast(StringType())))


For each dimension in `src_values`, the **Raw → Bronze → Silver → Gold** flow is executed:

1. **Raw → Bronze**  
   - Reads CSV files from the Raw layer.  
   - Deletes the previous directory in Bronze and overwrites it with Delta-formatted data.  

2. **Bronze → Silver**  
   - Reads the Bronze table.  
   - Applies the corresponding transformation function (`silver_transform_dates` or `silver_transform_times`).  
   - Saves the result as a table in the Silver schema (`silver_{dim}`).  

3. **Silver → Gold**  
   - Takes the Silver table.  
   - Deletes the previous Gold table if it exists.  
   - Creates the final dimensional table (`DimDates`, `DimTimes`) in the Gold schema.  


In [0]:
for dim in src_values:
    print(f"\n===== Procesando {dim.upper()} =====")

    pk_col = pk_map[dim]
    raw_path = f"{raw_base_path}/{dim}"
    bronze_path = f"{bronze_base_path}/{dim}"

    raw_df = (
        spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "false")
        .load(raw_path)
    )

    ############################# BRONZE ######################
    dbutils.fs.rm(bronze_path, recurse=True)
    raw_df.write.format("delta").mode("overwrite").save(bronze_path)

    ############################# SILVER ########################
    silver_table = f"{silver_schema}.silver_{dim}"
    bronze_df = spark.read.format("delta").load(bronze_path)

    if dim == "dates":
        bronze_df = silver_transform_dates(bronze_df)
    elif dim == "times":
        bronze_df = silver_transform_times(bronze_df)

    bronze_df.write.format("delta").mode("overwrite").saveAsTable(silver_table)

    ############################# GOLD ###################################
    gold_table = f"{gold_schema}.Dim{dim}"
    silver_df = spark.table(silver_table)
    
    spark.sql(f"DROP TABLE IF EXISTS {gold_table}")
    silver_df.write.format("delta").mode("overwrite").saveAsTable(gold_table)


===== Procesando DATES =====

===== Procesando TIMES =====
